In [ ]:
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow_examples.lite.model_maker.core.export_format import ExportFormat
from tensorflow_examples.lite.model_maker.core.task import image_preprocessing

from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader
from tflite_model_maker.image_classifier import ModelSpec

In [ ]:
# Retrieve the cassava plant disease dataset and splits into training, validation and test datasets

tfds_name = 'cassava'
# tfdsload function is used to load the dataset cassava as well as the splits to load 
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    name=tfds_name,
    split=['train', 'validation', 'test'],
    with_info=True,
    # loads the dataset in a format that can be used for supervised learning 
    as_supervised=True)
TFLITE_NAME_PREFIX = tfds_name

In [ ]:
# Construct the list of labels and loads the training and validation datasets

label_names = ds_info.features['label'].names

train_data = ImageClassifierDataLoader(ds_train,
                                       ds_train.cardinality(),
                                       label_names)
validation_data = ImageClassifierDataLoader(ds_validation,
                                            ds_validation.cardinality(),
                                            label_names)

# This selects the name of the model to use (mobilenet v3)

model_name = 'mobilenet_v3_large_100_224' 

# this maps specific model name to the URLs for their crresponding pre-trained weights to the tensorflow hub 
map_model_name = {
    'cropnet_cassava':
        'https://tfhub.dev/google/cropnet/feature_vector/cassava_disease_V1/1',
    'cropnet_concat':
        'https://tfhub.dev/google/cropnet/feature_vector/concat/1',
    'cropnet_imagenet':
        'https://tfhub.dev/google/cropnet/feature_vector/imagenet/1',
    'mobilenet_v3_large_100_224':
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
}

model_handle = map_model_name[model_name]

image_model_spec = ModelSpec(uri=model_handle)

In [ ]:
# Build the model by training the model with the training dataset

model = image_classifier.create(
    train_data,  # train_data: the training data, which should be a tf.data.Dataset object containing labeled images
    model_spec=image_model_spec,
    batch_size=128,  # the number of images to be processed in each batch during training.
    learning_rate=0.03, # the learning rate used by the optimizer during training.
    epochs=5, #the number of times to iterate over the entire training dataset during training.
    shuffle=True, # If True, all layers of the model are trainable
    train_whole_model=True,  # if False, only the classification layer is trainable.
    validation_data=validation_data)

# model.model.save('/tmp/my_model.h5')
# s3.upload_file('/tmp/my_model.h5', buket, object_key)

In [ ]:
import boto3
import io
ACCESS_ID = ""
ACCESS_KEY = ""

s3 = boto3.resource('s3', region_name='us-west-2',
         aws_access_key_id=ACCESS_ID,
         aws_secret_access_key= ACCESS_KEY)
bucket = s3.Bucket('planttest123')
bucket.upload_file('/tmp/my_model.h5', object_key)

In [ ]:
# This is a mapping dictionary of the disease codes to names

name_map = dict(
    cmd='Mosaic Disease',
    cbb='Bacterial Blight',
    cgm='Green Mite',
    cbsd='Brown Streak Disease',
    healthy='Healthy',
    unknown='Unknown')

[(name_map[x],x) for x in label_names]

In [ ]:
# Use the trained model to make predictions off the standard test dataset split from cassava 

test_data = ImageClassifierDataLoader(ds_test, ds_test.cardinality(),
                                      label_names)
model.predict_top_k(test_data)

In [ ]:
# Retrieve the image files from Amazon S3

import boto3
import io
# loading the the downloaded images as Numpy arrays 
import matplotlib.image as mpimg

ACCESS_ID = ""
ACCESS_KEY = ""

s3 = boto3.resource('s3', region_name='us-west-2',
         aws_access_key_id=ACCESS_ID,
         aws_secret_access_key= ACCESS_KEY)
bucket = s3.Bucket('planttest1234')
# storing 


bstream = io.BytesIO()
bucket.Object('image_1.JPG').download_fileobj(bstream)
img1 = mpimg.imread(bstream, format="JPEG")

bstream = io.BytesIO()
bucket.Object('image_2.JPG').download_fileobj(bstream)
img2 = mpimg.imread(bstream, format="JPEG"

In [ ]:
# Construct a new dataset using my image files 
input_1 = np.array((img1, img2))
input_2 = np.array((0, 0))

ds_test2 = tf.data.Dataset.from_tensor_slices((input_1, input_2))

#tf.data.Dataset.from_tensor_slices([img1, img2])
test_data2 = ImageClassifierDataLoader(ds_test2, ds_test2.cardinality(),
                                      label_names)

In [ ]:
# Show the images 
from PIL import Image as im

# Predict whether the images have disease
# Returns (label, probability)
labels = [(name_map[x[0][0]],x[0][1]) for x in model.predict_top_k(test_data2)]

i=0
for d in ds_test2:
    display(im.fromarray(d[0].numpy()))
    print(labels[i])
    i+=1

In [ ]:
CESS_ID = ""
ACCESS_KEY = ""
# plant tables are named like 'Plant_0, Plant_1, etc...
j = 0
plant_table_size = ['0', '1', '2', '3', '4', '5', '6', '7']
plant_table = str("Plant_" + plant_table_size[i])

dynamodb = boto3.resource('dynamodb', region_name='us-west-2',
         aws_access_key_id=ACCESS_ID,
         aws_secret_access_key= ACCESS_KEY)
table_name = 'Garden'
table = dynamodb.Table(table_name)
email_client = boto3.client("ses", region_name="us-west-2", 
                            aws_access_key_id=ACCESS_ID, 
                            aws_secret_access_key= ACCESS_KEY)

# Get the labels for the test data using the trained model
labels = [(name_map[x[0][0]]) for x in model.predict_top_k(test_data2)]
garden_id = 0
# Store the labels in the DynamoDB table
diseased = True
for label in labels:
    # Create an item to be inserted into the table
    if (label == "Healthy"):
        diseased = False
    key = "item" + str(i)
    i += 1
    
    item = {
        "PlantId": garden_id,
        "diseased": diseased
#         "probability": 
    }
        
    # Insert the item into the table
    table.put_item(TableName="Garden", Item=item)
    garden_id += 0
    
    body_data = str(garden_id) + " - " + label + " detected"
    body_page = """
        <html>
        <head></head>
        <body>
        <h2>Disease Detected - """ + label + """</h2>""" + """
        <p>Found in GardenId = """ + str(garden_id) + """
        </body>
        </html> """
    if diseased:
        try:
            email_client.send_email(Source="ttsega03@gmail.com", 
                                   Destination={
                                       "ToAddresses": [
                                           "mesteddy14@gmail.com"
                                       ]
                                   }, Message={
                                       "Subject": {
                                           "Data": "Disease Detected",
                                           "Charset": "UTF-8"
                                       },
                                       "Body": {
                                           "Text": {
                                               "Data": body_data,
                                               "Charset": "UTF-8"
                                           },
                                           "Html": {
                                               "Data": body_page,
                                               "Charset": "UTF-8"
                                           }
                                       }
                                   }
                                   )
        except Exception as e:
            print(e)
    garden_id += 1
    print("done!")